# Difficulties of Reinforcement Learning

1. Reward delay
2. Agent's action affect the subsequent data it receives


# Method categories

1. Policy-based. Learning an actor.
2. Value-based. Learning a critic.
3. Actor + Critic.Popular method: Asynchronous Advantage Actor-Critic (A3C).

# Learn an actor

Actor/Policy

$$Action = \pi(observation)$$

Three steps 
1. Define a set of function (Neural Network as Actor).
2. Goodness of function.
3. Pick the best function.


### Neural network as Actor

- Structure: Neural network
- Input: observatinon (image pixels etc.)
- Output: Actions (each action corresponds to a neuron in output layer), the probability of taking the actions.

The benefit of using network instead of lookup table is it can handle the unseen abservations.


### Goodness of actor.

- Given an actor $\pi_{\theta}(s)$ with network parameter $\theta$.
- Use the actor $\pi_{\theta}(s)$ to play the video game.

Episode:

- Start with observation $s_1$
- Machine decides to take $a_1$
- Machine obtains reward $r_1$
- Machine sees observation $s_2$
- Machine decides to take $a_2$
- Machine obtains reward $r_2$
- Machine sees observation $s_3$
- ......
- Machine decides to take $a_T$
- Machine obtains reward $r_T$

Total reward: $R_{\theta} = \sum_{t=1}^T r_t$.

However, even with the same actor, $R_{\theta}$ is different each time for the randomness in the actor and the game. Hence, our goal is to obtain the largest $\bar{R}_{\theta}$, which is the expected value of $R_{\theta}$ and evaluates the goodness of an actor $\pi_{\theta}(s)$.

We used a trajectory $\tau$ to represent an episode

$$\tau = \{s_1, a_1, r_1, s_2, a_2, r_2,\cdots,s_T, a_T, r_T\}$$

the total reward is 

$$R(\tau) = \sum_{t=1}^T r_t$$

If you use an actor to play the game, each $\tau$ has a probability to be sampled. The probability depends on actor parameter $\theta$, thus $P(\tau|\theta)$. The trajectory $\tau$ can be interpreted as a gaming process, $P(\tau|\theta)$ represents the probability of appearing of the gaming process given the actor that is governed by $\theta$. And gaming process consists of rewards. Our goal is to obtain the higher expected reward by adjusting the actor ($\theta$).

$$\bar{R}_{\theta} = \sum_{\tau} R(\tau)P(\tau|\theta)$$

Practically, we can use $\pi_{\theta}$ to play the game $N$ times to obtain $\{\tau^1, \tau^2, \cdots, \tau^N\}$, which can be view as samples that sampling from $P(\tau|\theta)$ $N$ times. Then the expected total reward is 

$$\bar{R}_{\theta} \approx \frac{1}{N}\sum_{n=1}^N R(\tau^n)$$

### Pick the best function

$$\theta^* = arg \underset{\theta}{\text{max}} \bar{R}_{\theta}$$

Gradient ascent

- start with $\theta^0$
- $\theta^1 \leftarrow \theta^0 + \eta \nabla \bar{R}_{\theta^0}$
- $\theta^2 \leftarrow \theta^1 + \eta \nabla \bar{R}_{\theta^1}$
- ......

The gradient takes the form

$$\begin{align*}
\nabla\bar{R}_{\theta} &= \nabla \sum_{\tau} R(\tau)P(\tau|\theta)\\
&= \sum_{\tau} R(\tau) \nabla P(\tau|\theta)\qquad \text{Thus, }R(\tau)\text{ do not have to be differentiable}\\
&= \sum_{\tau} R(\tau) P(\tau|\theta)\frac{\nabla P(\tau|\theta)}{P(\tau|\theta)}\\
&= \sum_{\tau} R(\tau) P(\tau|\theta)\nabla log P(\tau|\theta) \qquad \frac{dlog(f(x))}{dx} = \frac{1}{f(x)}\frac{df(x)}{dx}\\
&\approx \frac{1}{N}\sum_{n=1}^N R(\tau^n)\nabla log P(\tau^n|\theta) \qquad \text{N Samples}\\
&= \frac{1}{N}\sum_{n=1}^N R(\tau^n)\nabla log\Big(p(s_1^n)p(a_1^n|s_1^n,\theta)p(r_1^n, s_2^n|s_1^n, a_1^n)p(a_2^n|s_2^n,\theta)p(r_2^n,s_3^n|s_2^n,a_2^n)\cdots\Big)\qquad \text{probability of the nth trajectory}\\
&= \frac{1}{N}\sum_{n=1}^N R(\tau^n)\nabla log\Big(p(s_1^n)\prod_{t=1}^T p(a_t^n|s_t^n,\theta)p(r_t^n, s_{t+1}^n|s_t^n, a_t^n)\Big)\\
&= \frac{1}{N}\sum_{n=1}^N R(\tau^n)\nabla \Big(log p(s_1^n) + \sum_{t=1}^T log p(a_t^n|s_t^n, \theta)+\sum_{t=1}^T log p(r_t^n, s_{t+1}^n|s_t^n, a_t^n)\Big)\\
&= \frac{1}{N}\sum_{n=1}^N R(\tau^n)\nabla \sum_{t=1}^T log p(a_t^n|s_t^n, \theta)\\
&= \frac{1}{N}\sum_{n=1}^N \sum_{t=1}^T R(\tau^n)\nabla log p(a_t^n|s_t^n, \theta)
\end{align*}$$


### Equation analyse

The gradient of the expected reward $\bar{R}_{\theta}$ with respect to $\theta$ is given by

$$\nabla \bar{R}_{\theta} \approx \frac{1}{N}\sum_{n=1}^N \sum_{t=1}^T R(\tau^n)\nabla log p(a_t^n|s_t^n, \theta)$$

We can draw several action picking preference of actor from this equation.

1. The actor is clined to increase the probability of the action that gives the positive reward, whereas decrease the probability of the action that gives the negative reward.  
Extremely, by assuming the $\theta$ only control one action of a given state in a given trajactory, we transform the expected reward function to the following form
$$\bar{R}_{\theta} = \sum_{\tau } R(\tau)P(\tau|\theta) = \color{#cccccc}{\sum_{\tau \neq \tau^j} R(\tau)P(\tau|\theta) + \Big(p(s_1^j)\prod_{t\neq i}p(a_t^j|s_t^j)\prod_{t}p(r_t^j, s_{t+1}^n|s_t^j, a_t^j) \Big)}R(\tau^j)p(a_i^j|s_i^j, \theta) $$
The gradient ascent always helps us obtain higher reward $\bar{R}_{\theta}$. Moreover, trajectory reward $R(\tau^j)$ is a constant. Thus

$$\begin{array}{ll}
\text{If }R(\tau^j) \text{ is positive, }p(a_i^j|s_i^j) \text{ will increase}\\
\text{If }R(\tau^j) \text{ is negative, }p(a_i^j|s_i^j) \text{ will decrease}\\
\end{array}
$$

2. The actor prefers the actions that gives higher reward, i.e. increase the probability, which is guaranteed by the logarithm form.
$$\nabla \bar{R}_{\theta} \approx \frac{1}{N}\sum_{n=1}^N \sum_{t=1}^T R(\tau^n)\nabla log p(a_t^n|s_t^n, \theta) = \frac{1}{N}\sum_{n=1}^N \sum_{t=1}^T R(\tau^n) \frac{\nabla p(a_t^n|s_t^n, \theta)}{p(a_t^n|s_t^n, \theta)}$$
The denominator is regarded the normalization term such that eliminate the affect of the probability and increase the affect of the rewards.


### Add a baseline

In the enviromment that all the rewards are positive, the quantity of the probability of each action will increase, which is not the issue, because the probability is summed up to 1 such that the probability of the action that adds less quantity would decrease. However, practially, the probability are derived from sample data and there would exist the case that an action with high probability are not sampled, which would lead to improper probability decline over such actions.

To address this issue, a baseline that minus by the reward is introduced.

$$\nabla \bar{R}_{\theta} \approx \frac{1}{N}\sum_{n=1}^N \sum_{t=1}^T (R(\tau^n)-b)\nabla log p(a_t^n|s_t^n, \theta)$$

The baseline helps alleviate the problem of probability decline over unsampled high reward actions.


### Assign Suitable Credit

The action does not contributes to the former rewards but to the subsequent accumulated reward.

$$\nabla \bar{R}_{\theta} \approx \frac{1}{N}\sum_{n=1}^N \sum_{t=1}^T \left(\bbox[yellow]{\sum_{t'=t}^T r_{t'}^n}-b\right)\nabla log p(a_t^n|s_t^n, \theta)$$

Further away, the affect is weaker (add a discount).

$$\nabla \bar{R}_{\theta} \approx \frac{1}{N}\sum_{n=1}^N \sum_{t=1}^T \underbrace{\left(\bbox[yellow]{\sum_{t'=t}^T \gamma^{t'-t}r_{t'}^n}-b\right)}_{\text{Advantage Function } A^{\theta}(s_t, a_t)}\nabla log p(a_t^n|s_t^n, \theta)$$




### Alternative perspective

$$\nabla \bar{R}_{\theta} \approx \frac{1}{N}\sum_{n=1}^N \sum_{t=1}^T \underbrace{R(\tau^n)}_{\text{weight}}\underbrace{\nabla log p(a_t^n|s_t^n, \theta)}_{\text{classification}}$$


# Proximal Policy Optimization (PPO)

### On-policy v.s. Off-policy

- **On-policy**. Learn by playing game yourself.  
Conventionally, the policy gradient learning method has the following steps:  
  1. Use an agent to play a game $N$ times which correponds $N$ trajactories.
  2. The enviromment and the agent will also generate $N$ total rewards.
  3. For each time step in each trajectory, use their temporal state, action as well as the rewards to train the agent network.
  4. Use the gradient, which is the accumulated gradient of the N trajectories to update the network.  
  
  An update of network needs to play the game $N$ times, and after the network update, we have to play the game (sample the training data) again, which is too inefficient.
  
- **Off-policy**. Learn by watching others playing the game (or the archive video).  
The goal of off-policy is to use the sample which are sampled from $\pi_{\theta'}$ to train our current network coefficient $\theta$. $\theta'$ is fixed, so we can re-use the sample data.

### Importance Sampling

- Target: evaluate the expectation of $f(x)$ where $x$ derives from the distribution $p(x)$.
$$E_{x \sim p}[f(x)] = \int f(x)p(x) dx$$
- Approximation: 
$$E_{x \sim p}[f(x)]\approx \frac{1}{N}\sum_{i=1}^N f(x^i)$$
- If we only have $x^i$ sampled from distribution $q(x)$ instead of distribution $p(x)$
$$E_{x \sim p}[f(x)] = \int f(x)\frac{p(x)}{q(x)}q(x) dx = E_{x \sim q}\left[f(x)\frac{p(x)}{q(x)}\right]$$
Notice that the variance of the original and the current expectation is different.
$$\begin{array}{ll}
Var[X] &= E[X^2] &- &(E[X])^2\\
Var_{x\sim p}[f(x)] &= E_{x\sim p}[f(x)^2] &- &(E_{x\sim p}[f(x)])^2\\
Var_{x\sim q}\left[f(x)\frac{p(x)}{q(x)}\right] &= E_{x\sim p}\left[f(x)^2\frac{p(x)}{q(x)}\right] &- &(E_{x\sim p}[f(x)])^2\\
\end{array}$$
Consequently, in the case that the distribution $p$ and $q$ are fairly different, if our sample data is not enough, we may not obtain a similar expectation. As a result, it is better to constrain the distribution $p$ such that it is relatively close to the distribution $q$.

### On-policy to Off-policy

Using the inportance sampling method, the gradient of reward $\nabla \bar{R}_{\theta}$ turns out to be able to sampled from the distribution $p_{\theta'}(\tau)$

$$\begin{align*}
\nabla \bar{R}_{\theta} &= E_{\tau\sim p_{\theta}(\tau)}\left[R(\tau) \nabla log p_{\theta}(\tau)\right]\\
&= E_{\tau\sim p_{\theta'}(\tau)}\left[\frac{p_{\theta}(\tau)}{p_{\theta'}(\tau)}R(\tau) \nabla log p_{\theta}(\tau)\right]
\end{align*}$$

Advantage form

$$\begin{align*}
\nabla \bar{R}_{\theta} &= E_{(s_t, a_t)\sim \pi_{\theta}}\left[A^{\theta}(s_t, a_t) \nabla log p_{\theta}(a_t^n|s_t^n)\right]\\
&= \underbrace{E_{(s_t, a_t)\sim \pi_{\theta'}}\left[\frac{p_{\theta}(s_t, a_t)}{p_{\theta'}(s_t, a_t)}\bbox[yellow]{A^{\theta}(s_t, a_t)} \nabla log p_{\theta}(a_t^n|s_t^n)\right]}_{A^{\theta}(s_t, a_t) \text{ is actually } A^{\theta'}(s_t, a_t)\text{ because it's computed using the rewards from }\pi_{\theta'}}\\
&= \underbrace{E_{(s_t, a_t)\sim \pi_{\theta'}}\left[\frac{p_{\theta}(a_t|s_t)}{p_{\theta'}(a_t|s_t)}\bbox[yellow]{\frac{p_{\theta}(s_t)}{p_{\theta'}(s_t)}}A^{\theta}(s_t, a_t) \nabla log p_{\theta}(a_t^n|s_t^n)\right]}_{\text{eliminate }\frac{p_{\theta}(s_t)}{p_{\theta'}(s_t)} \text{by assuming the distribution of state has nothing to do with the agent}}\\
&= E_{(s_t, a_t)\sim \pi_{\theta'}}\left[\frac{p_{\theta}(a_t|s_t)}{p_{\theta'}(a_t|s_t)}A^{\theta}(s_t, a_t) \nabla log p_{\theta}(a_t^n|s_t^n)\right]\\
&= \nabla J^{\theta'}(\theta)
\end{align*}$$

where $J^{\theta'}(\theta)$ is an objective function and takes the form

$$J^{\theta'}(\theta) = E_{(s_t, a_t)\sim \pi_{\theta'}}\left[\frac{p_{\theta}(a_t|s_t)}{p_{\theta'}(a_t|s_t)}A^{\theta'}(s_t, a_t)\right]$$

which comes from the equation $\nabla f(x) = f(x)\nabla log f(x)$.

### TRPO / PPO /  PPO2


Recall that in the discussion of importance sampling, the distribution of $p$ and $q$ have to be similar in order to give similar expectation. Therefore while evaluating the gradient of the reward, the distribution of $\pi_{\theta}$ and $\pi_{\theta'}$ must be subjected to a constraint.

In Trust Region Policy Optimization (TRPO), we use constraint to address this issue.
$$J_{TRPO}^{\theta'}(\theta) = J^{\theta'}(\theta)\quad \text{ subject to } \quad KL(\theta, \theta')<\delta$$

In Proximal Policy Optimization (PPO), we use lagrange multiplier to address this issue, which is easier-implemented and  than TRPO.
$$J_{PPO}^{\theta'}(\theta) = J^{\theta'}(\theta) - \beta KL(\theta, \theta')$$

Both the two method use KL-divergence to measure the difference of distributions of $\pi_{\theta}$ and $\pi_{\theta'}$, i.e. $p_{\theta}(a_t|s_t)$ and $p_{\theta'}(a_t|s_t)$.

In PPO2, the distribution is cliped for constrain.

$$J_{PPO2}^{\theta'}(\theta) = E_{(s_t, a_t)\sim \pi_{\theta'}}\left[min\left(\frac{p_{\theta}(a_t|s_t)}{p_{\theta'}(a_t|s_t)}A^{\theta'}(s_t, a_t), clip\left(\frac{p_{\theta}(a_t|s_t)}{p_{\theta'}(a_t|s_t)}, 1-\epsilon, 1+\epsilon\right)A^{\theta'}(s_t, a_t)\right)\right]$$

### PPO implementation

1. Initial policy parameter $\theta^0$.
2. Using $\theta^k$ to interact with the envirnment to collect ${s_t, a_t}$ and compute advantage $A^{\theta^k}(s_t, a_t)$.
3. Substitute ${s_t, a_t}$ and $A^{\theta^k}(s_t, a_t)$ into the objective function $J_{PPO}^{\theta^k}(\theta)$, then use gradient ascent to optimize this function such that update $\theta$.
$$J_{PPO}^{\theta^k}(\theta) = J^{\theta^k}(\theta) - \beta KL(\theta, \theta^k)\qquad \text{where}\quad J^{\theta^k}(\theta)\approx \sum_{s_t, a_t}\frac{p_{\theta}(a_t|s_t)}{p_{\theta^k}(a_t|s_t)}A^{\theta^k}(s_t, a_t)$$
4. Repeat step 3, i.e. update $\theta$ several times.
5. Run step 2 again.

### Adaptive KL Penalty

- If $KL(\theta, \theta^k)> KL_{max}$, increase $\beta$.
- If $KL(\theta, \theta^k)< KL_{min}$, decrease $\beta$.


# Learn a critic

## State value function

- A critic does not directly determine the action.
- Given an actor $\pi$, it evaluates how good the actor is.
- State value function $V^{\pi}(s)$, indicates when using an actor $\pi$, the accumulated reward expects to be obtained after visiting state $s$.

In a word, the critic evaluate the reward of actor when facing some state.

### Method of evaluation

1. **Monte-Carlo (MC) based approach**. Suppose when using an actor $pi$, after seeing $s$, until the end of the episode, the expected cumulated reward is $G$, then the critic $V^{\pi}$, which is a network, should output $G$.
$$\begin{array}{ll}
\bbox[#aaaaff]{\underbrace{\Big[s_a\Big]}_{\text{input}}}\rightarrow \bbox[#ffaaaa]{\underbrace{\Big[V^{\pi}\Big]}_{\text{Network}}}\rightarrow V^{\pi}(s_a)\leftrightarrow \bbox[#aaaaff]{\underbrace{\Big[G_a\Big]}_{\text{target}}}\\
\bbox[#aaaaff]{\underbrace{\Big[s_b\Big]}_{\text{input}}}\rightarrow \bbox[#ffaaaa]{\underbrace{\Big[V^{\pi}\Big]}_{\text{Network}}}\rightarrow V^{\pi}(s_b)\leftrightarrow \bbox[#aaaaff]{\underbrace{\Big[G_b\Big]}_{\text{target}}}\\
\end{array}$$
This approach need us provides the accumulated reward which can only be obtained after finishing game episode. However, some games have very long episodes, so that delaying all learning until an episode's end is too slow. TD can address such issue.
2. **Temporal-difference (TD) approach**. The TD approach use the state of two subsequent states as input and the corresponding expected temporal-difference reward as output.

$$\left.\begin{array}{ll}
\bbox[#aaaaff]{\underbrace{\Big[s_t\Big]}_{\text{input}}} &\rightarrow \bbox[#ffaaaa]{\underbrace{\Big[V^{\pi}\Big]}_{\text{Network}}} &\rightarrow V^{\pi}(s_t)\\
\bbox[#aaaaff]{\underbrace{\Big[s_{t+1}\Big]}_{\text{input}}} &\rightarrow \bbox[#ffaaaa]{\underbrace{\Big[V^{\pi}\Big]}_{\text{Network}}} &\rightarrow V^{\pi}(s_{t+1})\\
\end{array}\right\} 
\quad \bbox[#dddddd]{\Big[- \Big]}
\rightarrow V^{\pi}(s_t) - V^{\pi}(s_{t+1})
\leftrightarrow \bbox[#aaaaff]{\underbrace{\Big[r\Big]}_{\text{target}}}$$

Obviously, learning a critic is a regression problem.

### MC v.s. TD

In MC, the target $G$ is actually a random variable which is composed of consecutive random variables correponding to various steps. The variance of $G$ is therefore very large.

In TD, the temporal-difference reward $r$ is also a random variable. But it has smaller variance. However, $V^{\pi}(s_{t+1})$ may be inaccurate.

### Example

- $s_a, r=0, s_b, r=0, $ END
- $s_b, r=1, $ END
- $s_b, r=1, $ END
- $s_b, r=1, $ END
- $s_b, r=1, $ END
- $s_b, r=1, $ END
- $s_b, r=1, $ END
- $s_b, r=0, $ END

In this episode. the expected reward after seeing $s_b$ is $V^{\pi}(s_b) = 3/4$. The expected reward after seeing $s_a$ is 
- Monte-Carlo: $V^{\pi}(s_a) = 0$
- Temporal-difference: $V^{\pi}(s_a) = V^{\pi}(s_b) + r = 3/4$


## State-action value function

State-action value function which is denoted by $Q^{\pi}(s, a)$, indicates that when using actor $\pi$, the cumulated reward expects to be obtained after taking $a$ at state $s$.

$$
\left.\begin{array}{ll}
\bbox[#aaaaff]{\big[s\big]} &\rightarrow \\
\bbox[#aaffaa]{\big[a\big]} &\rightarrow \\
\end{array}\right.
\underbrace{\bbox[#ffaaaa]{\Bigg[Q^{\pi}\Bigg]}}_{\text{network}}
\rightarrow Q^{\pi}(s, a)
$$

### Q-Learning

Now supposed that we are using the policy $\pi$ and facing the state $s$. If we have a better policy $\pi'$ that gives better critic than $\pi$, then we can replace $\pi$ with $\pi'$ to achive better critic, i.e. higher reward. In other words, if we can train the policy network $\pi$ to take the best action in order to achive higher reward, then our policy will become better. 

Mathematically, "better" is denoted by

$$ \begin{array}{ll}
& &V^{\pi'}(s) &\geqslant &V^{\pi}(s)\qquad &\text{for the spacific state }s\\
&i.e. &Q^{\pi'}(s, \pi'(s)) &\geqslant &Q^{\pi}(s, \pi(s))\qquad &\text{for the spacific state }s
\end{array}$$

Practically, the network is trained using various sampled state, thus, by training the network $Q^{\pi}(s, \pi(s))$, we can get a better network $Q^{\pi'}(s, \pi'(s))$ for all state

$$Q^{\pi'}(s, \pi'(s))\geqslant Q^{\pi}(s, \pi(s))\qquad \text{for all state }s$$

In the Q-Learning algorithm, when using policy $\pi$, we want the policy change to always takes it best action that gives the maximal $Q$ value. At that point, the policy $\pi$ has become another policy $\pi'$.

$$\pi'(s) = arg \underset{a}{\text{max}}Q^{\pi}(s, a)$$

In the perspective of neural network, we have

$$
\begin{array}{ll}
\text{Network:} & Q^{\pi}(s, a)\\
\text{Input:} & s,a\\
\text{Target:} & \underset{a}{\text{max}}Q^{\pi}(s, a)
\end{array}
$$



# Q-Learning Tips

### Training using TD approach

$$
\left.\begin{array}{ll}
\bbox[#aaaaff]{\big[s_t\big]} &\rightarrow \\
\bbox[#aaffaa]{\big[a_t\big]} &\rightarrow \\
\end{array}\right.
\underbrace{\bbox[#ffaaaa]{\Bigg[Q^{\pi}\Bigg]}}_{\text{network}}
\rightarrow Q^{\pi}(s_t, a_t)
\leftrightarrow \bbox[#ccccff]{\Big[r_t\Big]} + Q^{\pi}(s_{t+1}, \pi(s_{t+1}))
\leftarrow 
\underbrace{\bbox[#ffaaaa]{\Bigg[Q^{\pi}\Bigg]}}_{\text{network}}
\left.\begin{array}{ll}
\leftarrow &\bbox[#aaaaff]{\big[s_{t+1}\big]}  \\
\leftarrow &\bbox[#aaffaa]{\big[\pi(s_{t+1})\big]} \\
\end{array}\right.
$$

Although we can train this two networks, which are actually the same, simultaneously, it will be unstable because the target change too frequently. As a result, it is more often to fix the network on the right-hand side.

$$
\left.\begin{array}{ll}
\bbox[#aaaaff]{\big[s_t\big]} &\rightarrow \\
\bbox[#aaffaa]{\big[a_t\big]} &\rightarrow \\
\end{array}\right.
\underbrace{\bbox[#ffaaaa]{\Bigg[Q^{\pi}\Bigg]}}_{\text{network}}
\rightarrow Q^{\pi}(s_t, a_t)
\leftrightarrow \underbrace{\bbox[#ccccff]{\Big[r_t + Q^{\pi}(s_{t+1}, \pi(s_{t+1}))\Big]}}_{\text{target}}
\leftarrow 
\underbrace{\bbox[#bbbbbb]{\Bigg[Q^{\pi}\Bigg]}}_{\text{fix}}
\left.\begin{array}{ll}
\leftarrow &\bbox[#aaaaff]{\big[s_{t+1}\big]}  \\
\leftarrow &\bbox[#aaffaa]{\big[\pi(s_{t+1})\big]} \\
\end{array}\right.
$$

After updating the left-hand side network $N$ times, the right-hand side fixed network is replaced with the left-hand side network.



### Exploration

The Q function provide us the critic. If we only select the action that gives the highest critic, then the generated data, which will be used as the training data, is limited. E.g. suppose that the Q function is a not a network but a lookup table which is initialized to be zero. If at state s, the action $a_2$ is selected and the Q function returns 1, then $a_2$ will be always sampled because it gives the highest value.
$$
s\left\{
\begin{array}{ll}
a_1 & Q(s, a) = 0 &\text{Never explore}\\
a_2 & Q(s, a) = 1 &\text{Always sampled}\\
a_3 & Q(s, a) = 0 &\text{Never explore}
\end{array}
\right.
$$

#### Epsilon Greedy

$$a = \left\{
\begin{array}{ll}
arg \underset{a}{\text{max}} Q(s, a) & \text{with probability }1-\epsilon\\
random \text{otherwise}
\end{array}
\right.$$

where, $\epsilon$ would decay during learning.

#### Boltzmann Exploration

$$
p(a|s) = \frac{exp(Q(s, a))}{\sum_a exp(Q(s,a))}
$$

### Replay Buffer

Not only use the data from the interaction between $\pi$ and environment but also the previous data. All the data are maintained using a replay buffer, which is first in first out.

$$\bbox[#ffffbb]{
\left[\begin{array}{cc}
\pi \text{ interacts with }\\ 
\text{the environment}
\end{array}\right]}
\rightarrow 
\underbrace{\left[
\begin{array}{cc}
\vdots \\ 
\bbox[#aaddaa]{\text{ exp }} \\ 
\bbox[#aaffaa]{\text{ exp }} \\
\bbox[#ddaaaa]{\text{ exp }} \\
\bbox[#ffaaaa]{\text{ exp }} \\
\vdots
\end{array}
\right]}_{\text{replay buffer}}
\xrightarrow[]{\text{sample a batch}}
\bbox[#ddffdd]{
\Bigg[\text{Learning }Q^{\pi}(s, a)\Bigg]
}
$$
where the experience is denoted by $s_t, a_t, r_t, s_{t+1}$, is one interaction with the environment.

### Typical Q-Learning Algorithm

- Initialize Q-function $Q$, target Q-function $\hat{Q} = Q$
- In each episode
  - For each time step $t$
    1. Given state $s_t$, take action $a_t$ based on Q (epsilon greedy)
    2. Obtain reward $r_t$, and reach new state $s_{t+1}$
    3. Store $(s_t, a_t, r_t, s_{t+1})$ into buffer
    4. Sample $(s_i, a_i, r_i, s_{i+1})$ from buffer (ususlly a batch)
    5. Target $y = r_i+\underset{a}{\text{max}}\hat{Q}(s_{i+1}, a)$
    6. Update the parameters of $Q$ to make $Q(s_i, a_i)$ close to $y$ (regression)
    7. Every $C$ steps reset $\hat{Q} = Q$
    
    
### Double DQN

Q value is usually over-estimated, i.e. larger than the real reward.

For training Q function, we always select the action that gives the maximal Q function. Moreover, the Q function is estimated using network which exist deviation with respected to the real Q function, which leads to the over estimation on Q value.

$$Q(s_t, a_t) \leftrightarrow r_t + \underset{a}{\text{max}} Q(s_{t+1}, a)$$

In double DQN, we use two functions $Q$ and $Q'$. If $Q$ over-estimate $a$, so it is selected. $Q'$ would give it proper value. In contrast, the $Q'$ over-estimated actoin will not be selected by $Q$.

$$Q(s_t, a_t) \leftrightarrow r_t + Q'(s_{t+1}, arg\underset{a}{\text{max}}Q(s_{t+1}, a))$$


### Dueling DQN

$$
\begin{array}{ll}
\text{DQN:} &\rightarrow &\text{CNN} &\rightarrow &\text{FC} & &\rightarrow &Q(s, a)\\
\text{Dueling DQN:} &\rightarrow &\text{CNN} &\rightarrow &\left\{\begin{array}{ll}\text{FC}\\ \text{FC}\end{array}\right. & \left.\begin{array}{ll}\rightarrow\text{scalar }V(s)\\ \rightarrow\text{Vector }A(s,a) \end{array}\right\} &\rightarrow &Q(s, a) = A(s,a)+V(s)
\end{array}
$$

where $V(s)$ in the structure can be viewed as biases of state. In general, when implementing dueling DQN, we would add a layer normalization after $A(s,a)$ such that constrain the layer $A(s,a)$ and let the network to be more constributed by the scaler $V(s)$.


### Prioritized Reply

The data with larger TD error, harder to train, in previous training has higher probability to be sampled.


### Multi-step

Balance between MC and TD.

Saved trajectory.

$$\begin{array}{ll}
\text{MC:} &(s_t, a_t, r_t,\cdots, s_{END-1}, a_{END-1}, r_{END-1}, s_{END} )\\
\text{TD:} &(s_t, a_t, r_t, s_{t+1})\\
\text{Multi-step:} &(s_t, a_t, r_t, \cdots, s_{t+N}, a_{t+N}, r_{t+N}, s_{t+N+1}) 
\end{array}
$$

Q function target

$$Q(s_t, a_t) \leftrightarrow \sum_{t'=t}^{t+N} r_{t'}+\hat{Q}(S_{t+N+1}, a_{t+N+1})$$

### Noisy Net

The preceding Epsilon Greedy is an approch that add noise on action

$$a = \left\{\begin{array}{ll}
arg \underset{a}{\text{max}} Q(s, a) & \text{with probability }1-\epsilon\\
random &\text{otherwise}
\end{array}\right.$$

The Noisy Net approach inject noise into the parameters of Q-function at the beginning of each episode.
$$a = arg \underset{a}{\text{max}} \tilde{Q}(s, a) \qquad \text{where } Q(s,a) \xrightarrow[]{\text{Add Gaussian noise}} \tilde{Q}(s,a)$$

The noisy net is more commom on reality.

- Noise on action. In one episode, given the same state, the agent may takes different actions.
- Noise on parameter. In one episode, given the same(similar) state, the agent takes the same action, and try different action in the next episode. It is called state-dependent exploration, which explore in a consistent way.

### Distributional Q-function

$Q^{\pi}(s, a)$ denotes the expectation of the cumulated reward after seeing observation $s$ and taking $a$ by the actor $\pi$. However, different distributions may have the same expectation. This approach is proposed to output a distribution (histogram) from the network. Yet, this approach still select the action that gives the highest mean (expectation).

### Rainbow

Consist of all mentioned methods.


# Dilemmas on Q-learning

Action $a$ is a continuous vector. E.g. while driving a car, turn left and the degree.

$$a = arg \underset{a}{\text{max}} Q(s, a)$$



- Solution 1: Sample a set of actions: $\{a_1, a_2, \cdots , a_N\}$.
- Solution 2: Using gradient ascent to solve the optimization problem.
- Solution 3: Design a network to make the optimization easy.
$$s\rightarrow \underbrace{\bbox[#ffcccc]{\Bigg[Q^{\pi}\Bigg]}}_{\text{network}}
\left\{\begin{array}{ll}
\mu(s) &\text{vector}\\
\Sigma(s) &\text{matrix}\\
V(s) &\text{scalar}
\end{array}\right.$$
where the output Q value is 
$$Q(s, a) = -(a - \mu(s))^T\Sigma(s)(a-\mu(s))+V(s)$$
the optimal value of $a$ is 
$$\mu(s) = arg\underset{a}{\text{max}}Q(s, a)$$
- Solution 4: Don't use Q-learning in continuous cases.

# Actor-Critic

Asynchronous Advantage Actor-Critic (A3C)

- Asynchronous: 
- Advantage: 

### Policy Gradient

$$\nabla \bar{R}_{\theta} \approx \frac{1}{N}\sum_{n=1}^N\sum_{t=1}^{T_n}\underbrace{\left(\sum_{t'=t}^{T_n} \gamma^{t'-t}r_{t'}^n - b\right)}_{G_t^n\text{: obtained via interaction }b\text{: baseline}}\nabla log p_{\theta}(a_t^n|s_t^n)$$

### Q-Learning

